In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from toolz import concat, concatv, first, peek
from itertools import starmap

import json
from pathlib import Path
import gzip

In [3]:
from nbc_analysis.utils.debug_utils import runit
from nbc_analysis.utils.config_utils import get_config
from nbc_analysis.analysis.gen_profiles import main as gen_profile
from nbc_analysis.transforms.merge_video_ends import main as merge_video_ends

In [4]:
from cortex_common.types import EntityEvent, StringAttributeValue, ListAttributeValue

In [5]:
config = get_config(config_f='default',
                      overrides={'BATCHES_D': '/Users/wmcabee/Dropbox (Cognitive Scale)/NBC Analysis/data/NBC2/batches'})

>> created example config file '/Users/wmcabee/.config/nbc_analysis/config_example.yaml'
>> Using default config
>> WARNING: Overriding config file values with : {'BATCHES_D': '/Users/wmcabee/Dropbox (Cognitive Scale)/NBC Analysis/data/NBC2/batches'}


In [6]:
%%time
DATA = merge_video_ends(config_f=config)

>> Loading batches,batch_cnt=129,indir=/Users/wmcabee/Dropbox (Cognitive Scale)/NBC Analysis/data/NBC2/batches
>> Building index on {mpid,event_start_unixtime_ms}
>> Sorting index
>> end merge ve_events,outfile=/Users/wmcabee/DATA/NBC2/work/merged_ve_events.parquet,records=6265392
CPU times: user 1min 18s, sys: 10.1 s, total: 1min 28s
Wall time: 1min 23s


In [7]:
np.random.seed(7)
df= DATA[['show','genre']]
#df = df[:200]
df = df.reset_index()
GENRES = df.genre.drop_duplicates(keep='last').tolist()
SHOWS = df.show.drop_duplicates(keep='last').sort_values().tolist()

reader = filter(None, concatv(GENRES, SHOWS))
reader = concat(map(lambda x: x.split(' '),reader ))
reader = (x.lower() for x in reader if x.isalpha())
KEYWORDS = list(set(reader))


batch_id  \
mpid                 event_start_unixtime_ms                     
-9223313172072458017 1562290585268            ve_20190705_0001   
-9223308958333937889 1562107946230            ve_20190702_0018   
                     1562204304784            ve_20190704_0002   
                     1562348388478            ve_20190705_0011   
                     1562442345061            ve_20190706_0014   

                                                                                file  \
mpid                 event_start_unixtime_ms                                           
-9223313172072458017 1562290585268            NBC_App_20190705013950903090_84018.txt   
-9223308958333937889 1562107946230                NBC_20190702225317864436_79343.txt   
                     1562204304784                NBC_20190704014106617895_62036.txt   
                     1562348388478                NBC_20190705174453798848_62646.txt   
                     1562442345061                NBC_20190706194906736604_52661.txt   

                                              file_idx  \
mpid                 event_start_unixtime_ms             
-9223313172072458017 1562290585268                 591   
-9223308958333937889 1562107946230                8150   
                     1562204304784                 612   
                     1562348388478                6332   
                     1562442345061                7084   

                                                                  asof_dt  \
mpid                 event_start_unixtime_ms                                
-9223313172072458017 1562290585268            2019-10-01T22:07:26.419345Z   
-9223308958333937889 1562107946230            2019-10-01T21:36:59.725598Z   
                     1562204304784            2019-10-01T21:53:36.493592Z   
                     1562348388478            2019-10-01T22:16:25.493201Z   
                     1562442345061            2019-10-01T22:32:19.213351Z   

                                                  nbc_profile  \
mpid                 event_start_unixtime_ms                    
-9223313172072458017 1562290585268            Unauthenticated   
-9223308958333937889 1562107946230            Unauthenticated   
                     1562204304784            Unauthenticated   
                     1562348388478            Unauthenticated   
                     1562442345061            Unauthenticated   

                                                         mvpd event_name  \
mpid                 event_start_unixtime_ms                               
-9223313172072458017 1562290585268            Unauthenticated  Video End   
-9223308958333937889 1562107946230            Unauthenticated  Video End   
                     1562204304784            Unauthenticated  Video End   
                     1562348388478            Unauthenticated  Video End   
                     1562442345061            Unauthenticated  Video End   

                                                event_type platform  \
mpid                 event_start_unixtime_ms                          
-9223313172072458017 1562290585268            custom_event  Not Set   
-9223308958333937889 1562107946230            custom_event     roku   
                     1562204304784            custom_event     roku   
                     1562348388478            custom_event     roku   
                     1562442345061            custom_event     roku   

                                             data_connection_type  ...  \
mpid                 event_start_unixtime_ms                       ...   
-9223313172072458017 1562290585268                        Not Set  ...   
-9223308958333937889 1562107946230                        Not Set  ...   
                     1562204304784                        Not Set  ...   
                     1562348388478                        Not Set  ...   
                     1562442345061                        Not Set  ...   

                     

In [47]:
df = DATA[:200].reset_index()[['mpid', 'event_start_unixtime_ms']].sample(frac=1)
MPIDS = df.mpid
MPIDS_TS = df.event_start_unixtime_ms.astype(np.int)

In [48]:
def mock_scores(choices):
    count = np.random.randint(5,10)
    reader = zip(np.random.choice(choices,count, replace=False ),
              np.round(np.random.rand(count),5) )
    reader = filter(lambda x: x[0] not in {'', None}, reader)
    return sorted(reader, key=lambda x: -x[1])
  

In [49]:
def mock_profile(mpid, mpid_ts):
    return dict(mpid=mpid,
                genres=mock_scores(GENRES),
                shows=mock_scores(SHOWS),
                event_start_unixtime_ms=mpid_ts,
                #keywords=mock_scores(KEYWORDS),
               )

In [50]:
def write_dicts2json(inputs, outfile):
    reader = map(json.dumps, inputs)
    reader = map(lambda x: x+"\n", reader)
    reader = list(reader)


    with gzip.open(str(outfile), 'wt') as fh:
        fh.writelines(reader )
    print(f"wrote file={outfile},len={len(reader)}")

In [51]:
outfile = Path('.') / 'ex_from_cs1.json.gz'     
INPUTS = list(starmap(mock_profile, zip(MPIDS, MPIDS_TS)))
write_dicts2json(inputs=INPUTS, outfile=outfile)
print('example:')
INPUTS[0]

wrote file=ex_from_cs1.json.gz,len=200
example:


{'mpid': -9223307805084421834,
 'genres': [('Political', 0.73687),
  ('Talk and Interview', 0.60158),
  ('Celebrity and Gossip', 0.59718),
  ('Crime and Mystery', 0.48302),
  ('Drama', 0.40634),
  ('Reality and Game Show', 0.1352),
  ('Sports', 0.07847)],
 'shows': [('Days of our Lives', 0.9262),
  ('Cold Justice', 0.90393),
  ('Face Off', 0.86895),
  ('Everly', 0.69874),
  ('WAGS Miami', 0.51684),
  ('Law & Order: Trial By Jury', 0.32455),
  ('Deadly Class', 0.27187),
  ('The Voyager with Josh Garcia', 0.20236)],
 'event_start_unixtime_ms': 1562460862143}

## Convert to profile of 1 input format

In [46]:
def mock_po1( msg, attr, items):
    
    try: 
        items = ListAttributeValue(value=list(StringAttributeValue(value=value, weight=weight) for value, weight in items))
    
        event = EntityEvent(
              event=attr,
              entityId=str(msg['mpid']),
              entityType="nbc/Viewer",
              properties=items,
              meta= {},
              eventTime= msg['event_start_unixtime_ms'],
        )
        return dict(event)
    except Exception as e:
        print(items)
        raise
        
reader = ( (msg, attr, items) for msg in INPUTS for attr, items in msg.items() if attr not in {'mpid', 'event_start_unixtime_ms'})
reader = starmap(mock_po1, reader)
PROFILE_INPUTS = list(reader)
PROFILE_INPUTS[0]

{'event': 'genres',
 'entityId': '-9222835947932897233',
 'entityType': 'nbc/Viewer',
 'properties': {'value': [{'value': 'Lifestyle and Fashion',
    'weight': 0.49321,
    'context': 'cortex/attribute-value-string'},
   {'value': 'None',
    'weight': 0.47355,
    'context': 'cortex/attribute-value-string'},
   {'value': 'Political',
    'weight': 0.20565,
    'context': 'cortex/attribute-value-string'},
   {'value': 'Talk and Interview',
    'weight': 0.1627,
    'context': 'cortex/attribute-value-string'},
   {'value': 'Comedy',
    'weight': 0.03774,
    'context': 'cortex/attribute-value-string'}],
  'context': 'cortex/attribute-value-list'},
 'meta': {},
 'eventTime': 1570560605000}

In [20]:
outfile = Path('.') / 'ex_profile_input.json.gz'     
write_dicts2json(inputs=PROFILE_INPUTS, outfile=outfile)

wrote file=ex_profile_input.json.gz,len=6000


In [21]:
StringAttributeValue(value='x', weight=3)

StringAttributeValue(value='x', weight=3, context='cortex/attribute-value-string')